# TOPIC2VEC algorithm by using gensim and according to the second hint given by Gordon Mohr.  
(https://groups.google.com/forum/#!topic/gensim/BVu5-pD6910)


1. Vectorization of docs by using CountVectorizer (with or without tfidf) with no lemmatization
2. Latent Dirichlet Allocation 
3. Topic2Vec of the entire dataset (20 NewsGroups)   

It saves:
* the topic2vec model obtained

Changes:
1. Minor changes to be compatible with newer gensim library

In [51]:
# !pip install pyorient

In [52]:
import numpy as np; import pandas as pd; import matplotlib.pyplot as plt
%matplotlib inline
import codecs 
from glob import glob
import os
import pickle
import copy
import pyorient
import ast

In [53]:
from __future__ import print_function
from time import time, sleep
import string
import re
# random
from random import shuffle

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from sklearn.decomposition import LatentDirichletAllocation

In [54]:
from gensim import corpora, models, similarities

In [55]:
n_top_words = 20

## 1. IMPORTING DOCS FROM 20 NEWSGROUPS DATASET

In [56]:
from sklearn.datasets import fetch_20newsgroups
categories = ['comp.sys.ibm.pc.hardware',
'comp.sys.mac.hardware',
'comp.windows.x',
'rec.sport.baseball',
'rec.sport.hockey',
'sci.med',
'sci.space',
'soc.religion.christian']

n_topics = len(categories)

categories_source = {}

for cat in categories:
    categories_source[cat] = cat.replace('.', '_')

In [57]:
categories_source

{'comp.sys.ibm.pc.hardware': 'comp_sys_ibm_pc_hardware',
 'comp.sys.mac.hardware': 'comp_sys_mac_hardware',
 'comp.windows.x': 'comp_windows_x',
 'rec.sport.baseball': 'rec_sport_baseball',
 'rec.sport.hockey': 'rec_sport_hockey',
 'sci.med': 'sci_med',
 'sci.space': 'sci_space',
 'soc.religion.christian': 'soc_religion_christian'}

In [58]:
newsgroups_train = fetch_20newsgroups(subset='train',
                                      remove=('headers', 'footers', 'quotes'),
                                      categories=categories)

In [59]:
for i,j in categories_source.items():
    print(i,j)

rec.sport.hockey rec_sport_hockey
comp.windows.x comp_windows_x
soc.religion.christian soc_religion_christian
sci.space sci_space
rec.sport.baseball rec_sport_baseball
comp.sys.ibm.pc.hardware comp_sys_ibm_pc_hardware
sci.med sci_med
comp.sys.mac.hardware comp_sys_mac_hardware


#### TOTAL NUMBER OF DOC

In [60]:
n_docs = newsgroups_train.filenames.shape[0]
n_docs

4744

In [61]:
type(newsgroups_train)

sklearn.utils.Bunch

In [62]:
newsgroups_train.filenames

array(['/home/aimladmin/scikit_learn_data/20news_home/20news-bydate-train/sci.space/61065',
       '/home/aimladmin/scikit_learn_data/20news_home/20news-bydate-train/rec.sport.hockey/52618',
       '/home/aimladmin/scikit_learn_data/20news_home/20news-bydate-train/comp.windows.x/67032',
       ...,
       '/home/aimladmin/scikit_learn_data/20news_home/20news-bydate-train/rec.sport.hockey/52576',
       '/home/aimladmin/scikit_learn_data/20news_home/20news-bydate-train/soc.religion.christian/20809',
       '/home/aimladmin/scikit_learn_data/20news_home/20news-bydate-train/soc.religion.christian/20733'],
      dtype='<U96')

# 2. LDA to find the topic most-associated with each word

## 2.1 From Strings to Vectors

### WITH Lemmatization

### WITHOUT Lemmatization

In [63]:
t0 = time()
tf_vectorizer = CountVectorizer(encoding='utf-8', analyzer='word', stop_words='english',
                                ngram_range = (1,1), min_df = 2, token_pattern = '[a-zA-Z]{2,}').fit(newsgroups_train.data)
print("fit vectorizer without lemmatization done in %0.3fs." % (time() - t0))

fit vectorizer without lemmatization done in 0.700s.


In [64]:
CountVectorizer

sklearn.feature_extraction.text.CountVectorizer

In [65]:
tf_vectorizer

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=2,
        ngram_range=(1, 1), preprocessor=None, stop_words='english',
        strip_accents=None, token_pattern='[a-zA-Z]{2,}', tokenizer=None,
        vocabulary=None)

### Vectorization

In [66]:
# # get_feature_names() returns vocabulary
n_features = len(tf_vectorizer.get_feature_names())

In [67]:
n_features

19012

In [68]:
[tf_vectorizer.get_feature_names()[:10], tf_vectorizer.get_feature_names()[-10:]]

[['aa',
  'aaa',
  'aardvark',
  'aaron',
  'aas',
  'ab',
  'abandon',
  'abandoned',
  'abbott',
  'abbreviation'],
 ['zoomed',
  'zooming',
  'zorro',
  'zou',
  'zpixmap',
  'zterm',
  'zubov',
  'zupancic',
  'zupcic',
  'zyxel']]

In [69]:
newsgroups_train.data[0]

"Hmmm. I seem to recall that the attraction of solid state record-\nplayers and radios in the 1960s wasn't better performance but lower\nper-unit cost than vacuum-tube systems.\n\n\tMind you, my father was a vacuum-tube fan in the 60s (Switched\nto solid-state in the mid-seventies and then abruptly died; no doubt\nthere's a lesson in that) and his account could have been biased."

In [70]:
tf_docs = tf_vectorizer.transform(newsgroups_train.data)

In [71]:
tf_docs

<4744x19012 sparse matrix of type '<class 'numpy.int64'>'
	with 259893 stored elements in Compressed Sparse Row format>

### WITH TFIDF

tfidf_vectorizer = TfidfTransformer(sublinear_tf=False, use_idf = True).fit(tf_docs)
tfidf_docs = tfidf_vectorizer.transform(tf_docs)

## 2.2 LDA implementation

In [72]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

In [73]:
LatentDirichletAllocation

sklearn.decomposition.online_lda.LatentDirichletAllocation

In [74]:
print("Fitting LDA models with tf features, n_samples=%d and n_features=%d..."
      % (n_docs, n_features))
lda = LatentDirichletAllocation(n_topics=n_topics, max_iter=5,
                                learning_method='online', learning_offset=50.,
                                random_state=0)
t0 = time()
lda.fit(tf_docs)
print("done in %0.3fs." % (time() - t0))

Fitting LDA models with tf features, n_samples=4744 and n_features=19012...


/anaconda/envs/py35/lib/python3.5/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


done in 9.516s.


In [75]:
# tf_feature_names is vocabulary
print("\nTopics in LDA model:")
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)


Topics in LDA model:
Topic #0:
lssu gl cpu maria dma mask stormed dominating highlighting thirty belonged numlock mod wrist ik sgi rex indiana datahand cache
Topic #1:
like use just know does god people don time think problem drive edu new work good way window want thanks
Topic #2:
game year team games like season don think good win right just play players got didn did won hockey league
Topic #3:
food msg health disease cause patients diet lib medical diseases blood eat libxmu xmu foods day brain common effect doctor
Topic #4:
entry output com pts file edu period la buf entries pp rules power printf van oname build eof pt vs
Topic #5:
pitt gordon banks skepticism patients soon geb intellect jxp chastity dsl shameful cadre surrender candida pain edu yeast medical treatment
Topic #6:
space nasa launch orbit research center data earth shuttle satellite april lunar national moon year mission university information years science
Topic #7:
church catholic pope holy orthodox son schism autho

In [76]:
# row# of lda.components_ is topic#
# columns are the topic's unnormalized word weights (for all vocabulary words), in the sequence of the vocabulary word order
per_topic_distr_LDA = lda.components_
per_topic_distr_LDA.shape
#per_topic_distr_LDA.sum(axis=1)

(8, 19012)

# 3. TOPIC2VEC

In [77]:
# argmax example:
# >>> kkk
# array([[1, 2, 3],
#       [0, 4, 2]])
# >>> np.argmax(kkk, 0)
# array([0, 1, 0])
# >>> np.argmax(kkk, 1)
# array([2, 1])
#
# this will select the topic with the most word weight for each word in the vocabulary
# after this, we can easily lookup the best topic of each vocabulary word by 
# most_p_topic[word_voca_index] -> word's topic (0 -7 in this case) 
most_p_topic = np.argmax(per_topic_distr_LDA, axis=0)

In [78]:
# per_topic_distr_LDA

In [79]:
most_p_topic.shape

(19012,)

In [80]:
word_and_topic = zip(tf_feature_names, most_p_topic)
# word2topic_dict = {word : 'topic_' + np.array_str(topic) for word, topic in word_and_topic}
word2topic_dict = {word : 'topic_{}'.format(topic) for word, topic in word_and_topic}

In [81]:
# list the top 5 words and their belonged topics
from itertools import islice
list(islice(word2topic_dict.items(), 5))

[('positively', 'topic_1'),
 ('bottle', 'topic_6'),
 ('carumba', 'topic_4'),
 ('computer', 'topic_1'),
 ('cookiemonster', 'topic_4')]

## 3.1 Tokenization

In [82]:
def tokenizer(document):
    text = "".join([ch for ch in document if ch not in string.punctuation])
    text_list = text.split()
    normalized_text = [x.lower() for x in text_list]
    # Define an empty list
    nostopwords_text = []
    # Scan the words
    for word in normalized_text:
        # Determine if the word is contained in the stop words list
        if word not in ENGLISH_STOP_WORDS:
            # If the word is not contained I append it
            nostopwords_text.append(word)
    tokenized_text = [word for word in nostopwords_text if re.search('[a-zA-Z]{2,}', word)]
            
    return tokenized_text

In [83]:
def map_doc_to_topic(tokenized_text, prefix, doc_id_number, word2topic_dict):
    doc_to_topic_list = [prefix + '_' + str(doc_id_number)]
    # print('adding doc_to_topic header element {}'.format(doc_to_topic_list[0]))

    for word in tokenized_text:
        if word in word2topic_dict.keys():
            doc_to_topic_list.append(word2topic_dict[word])
        # else:
        #    print('{} not found in word2topic_dict.keys'.format(word))

    return doc_to_topic_list

In [84]:
from gensim.models.deprecated.doc2vec import LabeledSentence

In [85]:
class LabeledLineSentence_training(object):
    def __init__(self, sources, word2topic_dict):
        self.labels_list = word2topic_dict
        self.sources = sources
        flipped = {}
        # make sure that keys are unique
        for key, value in sources.items():
            if value not in flipped:
                flipped[value] = [key]
            else:
                raise Exception('Non-unique prefix encountered')
    
    def __iter__(self):
        print('len of sources is {}'.format(len(self.sources)))
        for source, prefix in self.sources.items():
            print(source)
            newsgroups_train_cat = fetch_20newsgroups(subset='train',
                                                      remove=('headers', 'footers', 'quotes'),
                                                      categories=[source])
            # print('len of newsgroups_train_cat is {}'.format(len(newsgroups_train_cat)))
            # (Pdb) newsgroups_train_cat.keys() -> 
            # dict_keys(['data', 'filenames', 'target', 'description', 'DESCR', 'target_names'])
            # import pdb; pdb.set_trace()
            for idx, doc in enumerate(newsgroups_train_cat.data):
                words_doc=tokenizer(doc)
                tags_doc = map_doc_to_topic(words_doc, prefix, idx, word2topic_dict)
                yield LabeledSentence(words = words_doc,
                                                     tags = tags_doc)
                
    def to_array(self):
        self.sentences = []
        print('len of sources is {}'.format(len(self.sources)))
        for source, prefix in self.sources.items():
            newsgroups_train_cat = fetch_20newsgroups(subset='train',
                                                      remove=('headers', 'footers', 'quotes'),
                                                      categories=[source])
            print('len of newsgroups_train_cat is {}'.format(len(newsgroups_train_cat)))
            # import pdb; pdb.set_trace()
            # (Pdb) type(newsgroups_train_cat) -> <class 'sklearn.utils.Bunch'>
            # (Pdb) type(newsgroups_train_cat.data) -> <class 'list'>
            # (Pdb) len(newsgroups_train_cat.data) -> 593
            # (Pdb) newsgroups_train_cat.data[0] -> document 1 strings, with newlines inside
            # (Pdb) newsgroups_train_cat.data[1] -> document 2 strings, with newlines inside
            # (Pdb) newsgroups_train_cat.target.shape -> (593,)
            # (Pdb) newsgroups_train_cat.target.max() -> 0
            for idx, doc in enumerate(newsgroups_train_cat.data):
                words_doc=tokenizer(doc)
                tags_doc = map_doc_to_topic(words_doc, prefix, idx, word2topic_dict)
                self.sentences.append(LabeledSentence(words = words_doc,
                                                     tags = tags_doc))
        return self.sentences
            
    def sentences_perm(self):
        shuffle(self.sentences)
        return self.sentences

## 3.1 Training

### Revisit parameters before training

In [86]:
categories_source

{'comp.sys.ibm.pc.hardware': 'comp_sys_ibm_pc_hardware',
 'comp.sys.mac.hardware': 'comp_sys_mac_hardware',
 'comp.windows.x': 'comp_windows_x',
 'rec.sport.baseball': 'rec_sport_baseball',
 'rec.sport.hockey': 'rec_sport_hockey',
 'sci.med': 'sci_med',
 'sci.space': 'sci_space',
 'soc.religion.christian': 'soc_religion_christian'}

In [87]:
list(islice(word2topic_dict.items(), 5))

[('positively', 'topic_1'),
 ('bottle', 'topic_6'),
 ('carumba', 'topic_4'),
 ('computer', 'topic_1'),
 ('cookiemonster', 'topic_4')]

### Tokenization

In [88]:
# For all input news group documents
#    For all sentences in that document
#        Generate gensim.models.deprecated.doc2vec.LabeledSentence
#            (words, tags with group name and word's topics)
it = LabeledLineSentence_training(categories_source, word2topic_dict)

#### Quote notes about LabeledSentence and TaggedDocument
1. LabeledSentence is an older, deprecated name for the same simple object-type to encapsulate a text-example that is now called TaggedDocument. 
2. Any objects that have words and tags properties, each a list, will do.
    - words is always a list of strings
    - tags can be a mix of integers and strings, but in the common and most-efficient case, is just a list with a single id integer, starting at 0.)

#### [Info about how to use Gensim doc2vec](https://medium.com/@mishra.thedeepak/doc2vec-in-a-simple-way-fa80bfe81104)
1. In this example it uses filename and doc label
2. And after the training it can print the vector of the file using its name

    ```
    docvec = d2v_model.docvecs[‘1.txt’] #if string tag used in training
    print docvec
    ```
3. Or to get most similar document with similarity scores using document-index

    ```
    similar_doc = d2v_model.docvecs.most_similar(14) 
    print similar_doc
    ```

In [89]:
# print the first new group's item #1
inspect_item = next(iter(it))
print(type(inspect_item))
print(inspect_item)
print(len(inspect_item.tags), len(inspect_item.words))
print(inspect_item.tags[:10], inspect_item.words[:10])

len of sources is 8
rec.sport.hockey
<class 'gensim.models.deprecated.doc2vec.LabeledSentence'>
LabeledSentence(['looks', 'like', 'edmonton', 'oilers', 'just', 'decided', 'european', 'vacation', 'spring', 'ranford', 'tugnutt', 'benning', 'manson', 'smith', 'buchberger', 'corson', 'playing', 'canada', 'podein', 'weight', 'playing', 'kravchuk', 'playing', 'russiansi', 'know', 'nagging', 'injuries', 'late', 'season', 'podein', 'interesting', 'casebecause', 'eligible', 'play', 'cape', 'breton', 'ahl', 'playoffs', 'like', 'kovalev', 'zubov', 'anderssonobviously', 'sather', 'pocklington', 'total', 'scrooges', 'makes', 'becertainly', 'case', 'theyve', 'massively', 'outclassed', 'paramount', 'new', 'york', 'rangers'], ['rec_sport_hockey_0', 'topic_2', 'topic_1', 'topic_2', 'topic_2', 'topic_1', 'topic_1', 'topic_6', 'topic_1', 'topic_2', 'topic_4', 'topic_4', 'topic_2', 'topic_4', 'topic_2', 'topic_4', 'topic_4', 'topic_2', 'topic_2', 'topic_2', 'topic_1', 'topic_2', 'topic_4', 'topic_2', 'top

/anaconda/envs/py35/lib/python3.5/site-packages/ipykernel_launcher.py:28: DeprecationWarning: Call to deprecated `LabeledSentence` (Class will be removed in 4.0.0, use TaggedDocument instead).


In [90]:
# type(models.Doc2Vec)
model = models.Doc2Vec(size=100, window=10, min_count=4, dm=1, dbow_words=1,
                              workers=50, alpha=0.025, min_alpha=0.025) # use fixed learning rate
model.build_vocab(it.to_array())

/anaconda/envs/py35/lib/python3.5/site-packages/gensim/models/doc2vec.py:359: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


len of sources is 8
len of newsgroups_train_cat is 6


/anaconda/envs/py35/lib/python3.5/site-packages/ipykernel_launcher.py:50: DeprecationWarning: Call to deprecated `LabeledSentence` (Class will be removed in 4.0.0, use TaggedDocument instead).


len of newsgroups_train_cat is 6
len of newsgroups_train_cat is 6
len of newsgroups_train_cat is 6
len of newsgroups_train_cat is 6
len of newsgroups_train_cat is 6
len of newsgroups_train_cat is 6
len of newsgroups_train_cat is 6


In [91]:
from tqdm import tqdm
for epoch in tqdm(range(10)):
    model.train(it.sentences_perm(), total_examples=model.corpus_count, epochs=1)
    model.alpha -= 0.002 # decrease the learning rate
    model.min_alpha = model.alpha # fix the learning rate, no decay

100%|██████████| 10/10 [03:05<00:00, 18.52s/it]


In [100]:
fname =  os.getcwd() # Prints the working directory
fname = fname + '/topic2vec_20NG_2_ndoc' + str(n_docs) + 'n_topic' + str(n_topics) + '.model'
model.save(fname)

### Show results
A quick info about [how to use gensim doc2vec model to query words by label vector or vice versa](https://github.com/RaRe-Technologies/gensim/issues/1397)

1. search words using word

    ```
    model.most_similar('word')
    # only similar words were returned but not labels
    ```
2. search label by label
    - use model.docvecs.most_similar to search for similar labels using labels
3. search words by label

    ```
    model.docvecs['label']
    model.similar_by_vector(label_vec)
    # only similar words were returned
    ```
4. search labels by word

    ```
    word_vec = model['word']
    model.docvecs.most_similar([word_vec])
    # returns similar labels
    ```

In [102]:
# load the model back
d2v_model = models.doc2vec.Doc2Vec.load(fname)

In [103]:
paragraphs_tag = d2v_model.docvecs.doctags
type(paragraphs_tag)
print(len(paragraphs_tag), list(islice(paragraphs_tag.items(),5)))

4752 [('sci_med_480', Doctag(offset=4060, word_count=19, doc_count=1)), ('comp_windows_x_490', Doctag(offset=1098, word_count=26, doc_count=1)), ('comp_sys_mac_hardware_455', Doctag(offset=4629, word_count=16, doc_count=1)), ('comp_sys_mac_hardware_487', Doctag(offset=4661, word_count=7, doc_count=1)), ('sci_space_236', Doctag(offset=2036, word_count=3192, doc_count=1))]


In [104]:
ragraphs_vector = d2v_model.docvecs.doctag_syn0
ragraphs_vector.shape

/anaconda/envs/py35/lib/python3.5/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `doctag_syn0` (Attribute will be removed in 4.0.0, use docvecs.vectors_docs instead).
  """Entry point for launching an IPython kernel.


(4752, 100)

In [105]:
d2v_model.docvecs.most_similar(positive = ['sci_space_96'])

[('rec_sport_baseball_389', 0.35365036129951477),
 ('rec_sport_baseball_378', 0.34690025448799133),
 ('comp_sys_ibm_pc_hardware_563', 0.3393045663833618),
 ('sci_med_524', 0.3304837942123413),
 ('comp_sys_mac_hardware_518', 0.32284557819366455),
 ('sci_med_125', 0.3165658116340637),
 ('rec_sport_hockey_94', 0.28935739398002625),
 ('comp_sys_mac_hardware_292', 0.289227157831192),
 ('rec_sport_hockey_203', 0.285552978515625),
 ('rec_sport_baseball_449', 0.28499162197113037)]

In [115]:
label_vec = d2v_model.docvecs['sci_space_96']
d2v_model.wv.similar_by_vector(label_vec)

[('lords', 0.39834630489349365),
 ('doesnt', 0.3908179998397827),
 ('darling', 0.3805291950702667),
 ('ahola', 0.35309669375419617),
 ('worry', 0.3530368208885193),
 ('destroyed', 0.3495197892189026),
 ('tale', 0.33816206455230713),
 ('cpus', 0.33736613392829895),
 ('lzone', 0.325408935546875),
 ('ears', 0.3231354355812073)]

In [114]:
topic_vec = d2v_model.docvecs['topic_0']
d2v_model.wv.similar_by_vector(topic_vec)

[('years', 0.8583284020423889),
 ('game', 0.8574833869934082),
 ('foremost', 0.7862314581871033),
 ('wouldnt', 0.7857553958892822),
 ('healing', 0.7753252983093262),
 ('important', 0.7587778568267822),
 ('mailing', 0.758521556854248),
 ('members', 0.7551657557487488),
 ('frequently', 0.7538352012634277),
 ('previous', 0.7529152631759644)]

In [113]:
word_vec = model['darling']
model.docvecs.most_similar([word_vec])

[('sci_space_96', 0.3805291950702667),
 ('rec_sport_hockey_236', 0.34010958671569824),
 ('rec_sport_baseball_592', 0.31272220611572266),
 ('soc_religion_christian_301', 0.29557496309280396),
 ('comp_windows_x_3', 0.29294127225875854),
 ('rec_sport_hockey_252', 0.2872934341430664),
 ('soc_religion_christian_265', 0.28440430760383606),
 ('sci_space_211', 0.2841663360595703),
 ('comp_sys_mac_hardware_86', 0.2825673818588257),
 ('soc_religion_christian_149', 0.28180262446403503)]

In [107]:
d2v_model.docvecs.n_similarity(['topic_0', 'topic_2'], ['topic_3', 'topic_4'])

0.644150725372538

In [108]:
d2v_model.docvecs.similarity('topic_0', 'topic_2')

0.3563553612509076